In [1]:
import json
import shutil
import cv2
import requests
import sys
import pandas as pd

LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40


def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr(image_path: str, appkey: str):
    """
    OCR api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()


    return requests.post(API_URL, headers=headers, files={"image": data})



img 폴더의 이미지 인식 및 결과값 출력

In [44]:

appkey='your_api'
tt=[]
for value in glob.glob("\img"+"\\*.*"):
    image_path=value
    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr(image_path, appkey).json()
    try:
        recognize =output['result'][0]['recognition_words'][0]
        if len(recognize)>1:
            #인식가능한 이미지만 따로저장
            #shutil.copy(image_path, "\img_answer"+"\\"+recognize+".png")
            
            print(recognize)
            tt+=[[value.split("\\")[-1].split("_")[0],recognize]]
        else:
            pass
    except:
        pass

en3
012445
013i6
01458
020155
02721
27924
030973
032462
033289
04876
øE4916
099043
F05036
113
12367
126711
12:817
1293
PO25T
31002
32195
149840
18R8
1E895t
16A453
165348
165800
70E03
17769
181303
184539
89125
19
199914
787
217,.00
215350
2567
22783
228742
458
226395
242558
242563
249615
250039
251041
2,9016
GR04
26&3966
260330
270354
Y822
974026
27505t
284242
84963
290948
34108
382219
308520
310325
3108
3E386
814679
31839
327979
R28767
30199
33710
E52292
3:?413
369620
SS6t
E80683
E8883
290501
332992
39,356
403762
403571
41R0
118286
421807
1303
BAC0
140473
784
449G3
4R48
458778
1194
645
4781
4&747
490979
311
ø9A714
+95026
497F65
504904
50573
507329
F08996
E09357
513034
53247
5B419
514374
515881
4ø16543
5,9090
Foti7
B802
597494
SIO1g7
284
543.6T
5ø0900
Si80
554672
F61995
F62661
SG6:87
53742
58]686
58543
502430
507007
FO9RT3
602078
6O+86
611L.60
512301
61450
616587
E182
62T
631441
27972
641203
642209
GA6407
6P06
63481
650435
659608
668A29
ø65843
669139
EGM9G9
8]2
6T7952
678T.0
5,8978
6879

In [47]:
df=pd.DataFrame(tt)

정확도 출력

In [4]:
len(df.loc[df[0] ==df[1]])/len(df)*100

28.01556420233463